In [14]:
# Import necessary libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Define constants
DATADIR = "C:/brain_tumor_dataset"
CATEGORIES = ["no", "yes"]
IMG_SIZE = 100

# Function to create data
def create_data():
    data = []
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        print(f"Checking directory: {path}")  # Debug statement
        if not os.path.exists(path):
            print(f"Directory does not exist: {path}")  # Debug statement
            continue
        
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_path = os.path.join(path, img)
                img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img_array is None:
                    print(f"Failed to load image: {img_path}")  # Debug statement
                    continue
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                data.append([new_array, class_num])
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")  # Debug statement
    return data

# Load data
data = create_data()
if len(data) == 0:
    raise ValueError("No images were loaded. Please check your data directory.")

random.shuffle(data)

# Separate features and labels
X = []
y = []
for features, label in data:
    X.append(features)
    y.append(label)

# Convert lists to numpy arrays and reshape
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)

# Normalize pixel values
X = X / 255.0

# Binarize labels
lb = LabelBinarizer()
y = lb.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# Fit the model
model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=10, validation_data=(X_test, y_test))

# Save the model
model.save('brain_tumor_detector.h5')
print("Model saved as brain_tumor_detector.h5")

# Load the model
loaded_model = load_model('brain_tumor_detector.h5')
print("Model loaded from brain_tumor_detector.h5")

# Evaluate the model on the test set
y_pred = loaded_model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Print confusion matrix and classification report
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Function to prepare a single image for prediction
def prepare_image(filepath):
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"File does not exist: {filepath}")
    
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    if img_array is None:
        raise ValueError(f"Failed to load image: {filepath}")
    
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1) / 255.0

# List of image paths to automate prediction for multiple images
image_paths = [
    'C:/brain_tumor_dataset/yes/Y1.jpg',
    'C:/brain_tumor_dataset/yes/Y2.jpg',
    'C:/brain_tumor_dataset/yes/Y3.jpg',
   
    
    'C:/brain_tumor_dataset/yes/Y7.jpg',
    
    'C:/brain_tumor_dataset/yes/Y10.jpg',
    'C:/brain_tumor_dataset/yes/Y11.jpg',
   
    
    'C:/brain_tumor_dataset/no/no18.jpg',
    'C:/brain_tumor_dataset/no/No14.jpg',
    
    'C:/brain_tumor_dataset/no/No19.jpg',
    'C:/brain_tumor_dataset/no/No20.jpg',
    
]

# Iterate over each image path and make predictions
for image_path in image_paths:
    try:
        sample_image = prepare_image(image_path)
        prediction = loaded_model.predict(sample_image)
        result = 'Tumor' if prediction[0][0] > 0.5 else 'No Tumor'
        print(f"Prediction for {image_path}: {result}")
    except (FileNotFoundError, ValueError) as e:
        print(e)


Checking directory: C:/brain_tumor_dataset\no
Checking directory: C:/brain_tumor_dataset\yes
Epoch 1/10


C:\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 123ms/step - accuracy: 0.5576 - loss: 0.6957 - val_accuracy: 0.6863 - val_loss: 0.5968
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - accuracy: 0.6898 - loss: 0.6051 - val_accuracy: 0.7647 - val_loss: 0.5422
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - accuracy: 0.7681 - loss: 0.5455 - val_accuracy: 0.8039 - val_loss: 0.5145
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - accuracy: 0.7354 - loss: 0.5217 - val_accuracy: 0.8039 - val_loss: 0.4881
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 0.7477 - loss: 0.5358 - val_accuracy: 0.8627 - val_loss: 0.4917
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - accuracy: 0.7847 - loss: 0.4786 - val_accuracy: 0.7843 - val_loss: 0.4612
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.7519 - loss: 0.5196 - val_accuracy: 0.7843 - val_loss: 0.5435
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 0.7058 - loss: 0.5836 - val_accuracy: 0.8431 - val_loss: 0.4690
Epoch 9/10

Model saved as brain_tumor_detector.h5


Model loaded from brain_tumor_detector.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Confusion Matrix:
[[12  9]
 [ 1 29]]

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.57      0.71        21
           1       0.76      0.97      0.85        30

    accuracy                           0.80        51
   macro avg       0.84      0.77      0.78        51
weighted avg       0.83      0.80      0.79        51

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Prediction for C:/brain_tumor_dataset/yes/Y1.jpg: Tumor
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Prediction for C:/brain_tumor_dataset/yes/Y2.jpg: Tumor
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Prediction for C:/brain_tumor_dataset/yes/Y3.jpg: Tumor
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Prediction for C:/brain_tumor_dataset/yes/Y7.jpg: Tumor
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Prediction for C:/brain_tumor_dataset/yes/Y10.jpg: Tumor
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Prediction for C:/brai